# Load LoRA adapters

In [1]:
import torch
if torch.cuda.get_device_capability() < (7, 5):
    raise ValueError(f"You got a GPU with capability {torch.cuda.get_device_capability()}, need at least (7, 5)")
else: print("OK")

OK


In [2]:
%pip install -q bitsandbytes==0.37.0 transformers==4.27.4 datasets==2.7.0 accelerate==0.18.0 loralib==0.1.1 peft==0.3.0.dev0

Note: you may need to restart the kernel to use updated packages.


#### Load base model

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

CACHE_DIR = '/media/tfsservices/DATA/NLP/cache/'
MODEL_NAME = "EleutherAI/gpt-neox-20B"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,        # bitsandbytes lib required (convert the loaded model into mixed-8bit quantized model.)
    device_map='auto',
    torch_dtype=torch.float16,
    cache_dir=CACHE_DIR,      # path to a directory in which a downloaded pretrained model
    )
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 46/46 [00:27<00:00,  1.68it/s]


In [4]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = stops
        self.ENCOUNTERS = encounters

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        stop_count = 0
        for stop in self.stops:
            stop_count = (stop == input_ids[0]).sum().item()

        if stop_count >= self.ENCOUNTERS:
            return True
        return False

In [10]:
stop_words_ids1 = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in ["###"]]
stopping_criteria1 = StoppingCriteriaSub(stops=stop_words_ids1, encounters=1)

stopping_criteria = StoppingCriteriaList([stopping_criteria1])

# stop_words_ids2 = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in ["A:"]]
# stopping_criteria2 = StoppingCriteriaSub(stops=stop_words_ids2, encounters=2)

# stopping_criteria = StoppingCriteriaList([stopping_criteria1, stopping_criteria2])

In [11]:
def remove_promt(generated: str, prompt: str):
    '''
    Remove original promt and specific signs from the model output.
    '''
    prompt_length = len(prompt)
    output = generated[prompt_length:]
    output = output.replace("###", "")
    return output

In [12]:
def generate(model, prompt: str, max_length=250):

    batch = tokenizer(prompt, return_tensors="pt").to('cuda')

    with torch.no_grad():
    # with torch.cuda.amp.autocast():
        
        outputs = model.generate(**batch,
                                # early_stopping=True, 
                                # num_beams=6,
                                # num_beam_groups = 3,
                                # no_repeat_ngram_size = 4,
                                # temperature=0.9,
                                top_k=10,
                                # top_p=0.8,
                                # penalty_alpha=0.6,
                                # repetition_penalty = 1.05,
                                do_sample=True,

                                max_length = max_length,
                                pad_token_id=tokenizer.eos_token_id,
                                stopping_criteria=stopping_criteria,
                                
                                # use_cachе=True,
                                # output_scores=True,
                                # return_dict_in_generate=True).detach()
        )
        generated = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
        generated = remove_promt(prompt=prompt, generated=generated)
        print(f"tokens: {len(generated.split())}")
        print(f"text:\n{generated}")

In [ ]:
def generate_armm_listing(model):

    adapter_name = "adapters/gpt-neox-20B-armm_list"
    model = PeftModel.from_pretrained(model, adapter_name, device_map={"":0})

    generate(model, "Q: Generate\n A:", max_length=512)
    

In [ ]:
def generate_armm_requirement(model):

    adapter_name = "adapters/gpt-neox-20B-armm_req"
    model = PeftModel.from_pretrained(model, adapter_name, device_map={"":0})

    generate(model, "Q: Generate\n A:", max_length=512)

In [ ]:
def generate_wm_resp_listing(model):

    adapter_name = "adapters/gpt-neox-20B-wm_list_resp"
    model = PeftModel.from_pretrained(model, adapter_name, device_map={"":0})

    generate(model, "Q: Generate response\n A:", max_length=512)

In [8]:
def generate_wm_resp_purchase(model):

    # adapter_name = "adapters/gpt-neox-20B-wm_purch_resp"
    adapter_name = "training/epoch-4.7"
    model = PeftModel.from_pretrained(model, adapter_name, device_map={"":0})

    # generate(model, "Q: Generate response\n A:", max_length=512)
    generate(model, "Q: Generate\n A:", max_length=512)

#### Generate ARMM: listing 

In [34]:
generate_armm_listing(model)

tokens: 76
text:
 Hi {Client},

Our client is seeking a {Model} {Category} {MakeModel}. We have a complete {SpareParts} available for the sale. The seller is asking {Price} {SpareParts}.

They are hoping for the same day deal.

We have a complete {SpareParts} available to complete the purchase at this time.

They have released this unit to their supplier and they are hoping to move it soon.

Let me know if you have any questions.

Thank you.

{EquipDetails}

Best Regards,
{Signature}



In [35]:
generate_armm_listing(model)

tokens: 83
text:
 Hi {Client},

We have a new {MakeModel} {Category} {Category} available for sale. It is currently being used by the user but they are looking to release it from the facility before the end of the year. 

{EquipDetails}

This unit is in {MakeModel} {Category}. {Qty} units were made from the same tooling for {AipClient}. Please check their {SpareParts} for additional details. {Timeline}

Please review this and let me know if you have any interest. Please let me know if you have additional requests. {Signature}



In [36]:
generate_armm_listing(model)

tokens: 91
text:
 Hi {Client},

I hope you are well.  I have a client that is selling a {MakeModel}.  The seller wants to make a quick offer on this item before it goes for auction.  This item is only released to the market for {Qty} {Timeline}, and is still in the cleanroom in a clean and working condition.  The seller is only asking {Price} as a complete package.  Please see the photos below.  {EquipDetails}

Please let me know if you have any questions and would like to have a discussion on this unit.

{Signature}



In [37]:
generate_armm_listing(model)

tokens: 53
text:
 Hi {Client},

We have a used A4 {MakeModel} {Category} for sale at {Price} only, the seller said the seller didn t want to use the tool anymore. This {Category} has been used only for {Timeline} in complete working condition.

Please review the photos and let me know if you are intersted.
{EquipDetails}

{Signature}



In [38]:
generate_armm_listing(model)

tokens: 64
text:
 Hi {Client},

I have some systems which I need to move. 

The system {MakeModel} was installed last year and the owner decided {Qty}, {Price}. It was a new system when it was installed, with only a few systems installed in the {EquipCategory}, so there are still a lot of options available for you. If you are interested, please let me know ASAP. 

{EquipConfig}

{Signature}



#### Generate ARMM: requirement 

In [22]:
generate_armm_requirement(model)

tokens: 110
text: 

Hi {Client},

We have a need to purchase {MakeModel} for our client {SpareParts}, they need {WaferSize} {Category}. They will purchase it as well if they see it. 

Our team will be arriving {Timeline} for the purchase inspection. 

Our buyer would prefer to make a deal with the seller. 

If you have this tool available and you are looking to get rid of it, please kindly let me know your offer as soon as possible.

I'm sorry that my client's budget is tight and we are looking for the tool in this week.  If you are interested in this, please kindly contact me.  My client's budget is {Timeline}.

Thank you,
{Signature}



In [39]:
generate_armm_requirement(model)

tokens: 62
text:
 Hello {Client},

We're seeking for a {Qty} {MakeModel} {Category} {WaferSize} {Timeline} for our client. This tool is required by the {MakeModel} to complete a project for a {MakeModel} client.

Our client will also consider other options, like {MakeModel}, {Category}, {Category}.

Our client would like to hear from you as soon as we can for a quick response.

Thank you.

Best regards,
{Signature}



In [40]:
generate_armm_requirement(model)

tokens: 99
text:
 Hi {Client},

We have received a purchase request of {Qty} {MakeModel} system for a customer, who is planning to expand his existing facility to a new one. 

They would like to have a fully functional system as the existing system is still in working condition.

Their preferred timeline to get their approval is {Timeline}. They are planning to make an offer within this time frame.

I was wondering if you have any for sale, or if you have any other similar tools for sale. 

Please feel free to let me know, I can send you photos. 

Best regards, {Signature}



In [41]:
generate_armm_requirement(model)

tokens: 91
text:
 Hi  {Client},

I have a {MakeModel} {Category} which is currently being used in my {SpareParts} business. I am in the process of reviewing our existing tools and would like to purchase this tool as well. It will be used in our {Timeline} {TimelineA} {TimelineB} for my client's toolset.

It is preferred if you have the tool in hand. Please reply if you have one for sale. If you have one for sale, please also let me know your price range as well as any additional details regarding it.

Best regards,
{Signature}



In [42]:
generate_armm_requirement(model)

tokens: 51
text:
 Dear {Client},
I am working with a client looking to purchase a {MakeModel} for their facility. Do you have any of these units for sale at the moment? 

{WaferSize}

If you do, please let me know so that I could schedule a site visit as soon as possible. 

Best regards,
{Signature}



#### Generate response for WM: New Listing 

In [43]:
generate_wm_resp_listing(model)

tokens: 43
text:
 Hi {Client}, 
 Thank you for reaching out. I'm {Trader} from CAE. Please send me the photos and configurations for the {Category} you want to sell. If you have any other {Category} you are interested in, please let me know.
 Thank you
 Best

A: 




In [44]:
generate_wm_resp_listing(model)

tokens: 25
text:
 Hi {Client}, 
 
 Thanks for reaching out. Could you please send me some photos or a photo and configuration list for this unit? 
 
 Best regards,
 {Signature}



In [45]:
generate_wm_resp_listing(model)

tokens: 33
text:
 Hi {Client}, 

I received your message about selling your {MakeModel}. 
  
I am very interested in it. Do you have pictures of all the {MakeModel} you are willing to sell? 

Thanks. 

Best Regards,
{Signature}



In [46]:
generate_wm_resp_listing(model)

tokens: 37
text:
 Hi {Client},

I received your request for a sale on these units. I've attached photos for you as well as an item listing. I'll be available after {Date} to take the sale off the market.

Thanks,
{Signature}



In [47]:
generate_wm_resp_listing(model)

tokens: 51
text:
 Dear {Client}, 
  
Good morning. This is {Trader} from CAE, I am contacting you on behalf of {Buyer}. We have your {MakeModel} {Category} {Category} system, we are interested in selling this machine. Could you send us photos of it and let me know your asking price?
  
Looking forward to your response,

{Signature}



#### Generate response for WM: Purchase Inquiry 

In [13]:
generate_wm_resp_purchase(model)

tokens: 59
text:
 Hello,

I received your message about the webcam and I was able to locate the webcam. This is a photo of the unit and its configuration. The price includes the unit and the web setup fee. We do not have the unit in-house, but we can order it for you. Please let me know your timeline. 

{EquipDetails}

{Signature}
#



In [14]:
generate_wm_resp_purchase(model)

tokens: 23
text:
 Hi {Client}, 

Good day. 
Please find my web inquiry below. Do you have any further information or requirement for this item? 

{EquipDetails}

{Signature}



In [15]:
generate_wm_resp_purchase(model)

tokens: 94
text:
 {Client}, 

{EquipDetails}

Good morning. How are you, hope everything is good with you. This is {Trader} from CAE. I am contacting you in regards to one of our suppliers. We received an inquiry in our system and would like to know if you are still in the market of {MakeModel}, {Category}. 
Let me know what you are looking for and also if it's an offer or you would like to see it. If you do not currently own any of this model/category let me know and I will provide you with some information.

{Signature}



In [16]:
generate_wm_resp_purchase(model)

tokens: 55
text:
 Hello {Client}, 

Please note we received your request, 

This item is for sale. 
{EquipDetails} 
We have a limited number of this model. Please let me know the timeline that you are working with, and if you have other model you would want to trade in for a newer one, or purchase as-is. 

Thank you.

{Signature}



In [52]:
generate_wm_resp_purchase(model)

tokens: 47
text:
 {Client},
 This is {Trader} from CAE. I have received your inquiry about {MakeModel}
 Do you currently run this model? If you have any questions I'd be happy to help you. Please let me know the details below and we will follow up ASAP.

{EquipDetails}

Best Regards,
{Signature}



#### The End